In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import math
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

<!-- ##### Object for individual customer
- Calculates net-load of each customer. Inputs: each customer's demand, microgeneration forecasts, initial SOC, battery status (0/1)
- Energy management at the household -->

In [2]:
class Customer: # Customer Energy Management
    def __init__(self,hh_demand,pvsize,hh_SOC,batt_size,Pr_grid,fit):
        self.hh_demand=hh_demand
        self.hh_gen=Pg.iloc[t].values*pvsize
        self.hh_SOC=hh_SOC
        self.batt_size=batt_size
        self.Pr_fit=fit
        self.Pr_grid=Pr_grid
        self.E=batt_size
        self.Pbmax=batt_size*Crate
        
        self.hh_demand = self.price_responsive_load()

        if self.batt_size>0: #battery management
            self.battery=self.battery_management()
            self.Pgrid=self.battery[0]
            self.SOC2=self.battery[1]
            self.Pb=self.battery[2]
        else: #Energy management in absence of battery
            self.Pgrid=(self.hh_demand-self.hh_gen)
            self.SOC2=0
            self.Pb=0

    def price_responsive_load(self):
        if self.Pr_grid >= Pr_th_s:
            self.hh_demand = 0.5 * self.hh_demand + 0.5 * self.hh_demand * math.exp(-8 * (Pr_th_s - Pr_th_b))
        elif self.Pr_grid >= Pr_th_b and self.Pr_grid <= Pr_th_s:
            self.hh_demand = 0.5 * self.hh_demand + 0.5 * self.hh_demand * math.exp(-8 * (self.Pr_grid - Pr_th_b))
        else:
            self.hh_demand = 0.5 * self.hh_demand + 0.5 * self.hh_demand * math.exp(-8 * (Pr_th_b - Pr_th_b))
        return self.hh_demand

    def battery_management(self):
        if self.hh_gen>self.hh_demand:
            if self.Pr_fit>=Pr_th_s: #sell from the battery and the surplus gen 
                self.Pbcal=(self.hh_SOC-SOC_min)*(self.E*nd)/dT #battery discharging
                if self.Pbcal>=self.Pbmax:
                    self.Pb=-self.Pbmax
                else:
                    self.Pb=-self.Pbcal
            elif self.Pr_grid>Pr_th_b: #dead band
                self.Pb=0
            else:
                self.Pbcal=(SOC_max-self.hh_SOC)*self.E*nc/dT #battery charging if price < buying threshold
                if self.Pbcal>=self.Pbmax:
                    self.Pb=self.Pbmax
                else:
                    self.Pb=self.Pbcal
        else:
            if self.Pr_grid<=Pr_th_b:
                self.Pbcal=(SOC_max-self.hh_SOC)*self.E/(nc*dT) #battery charging
                if self.Pbcal>=self.Pbmax:
                    self.Pb=self.Pbmax
                else:
                    self.Pb=self.Pbcal
            elif self.Pr_grid<=Pr_th_s:
                self.Pb=0
            else:
                self.Pbcal=(self.hh_SOC-SOC_min)*(self.E*nd)/dT #discharging
                if self.Pbcal>=self.Pbmax:
                    self.Pb=-self.Pbmax
                else:
                    self.Pb=-self.Pbcal
        self.Pgrid=(self.hh_demand-self.hh_gen)+self.Pb
        self.SOCend=self.hh_SOC + ((self.Pb*nc)*(dT/self.E) if self.Pb>=0 else (self.Pb/nd)*(dT/self.E))
        return self.Pgrid, self.SOCend, self.Pb, self.hh_gen, self.hh_demand

In [3]:
class CustomerOffgrid:
    def __init__(self,hh_demand,pvsize,hh_SOC,batt_size):
        self.hh_demand=hh_demand
        self.hh_gen=Pg.iloc[t].values*pvsize
        self.hh_SOC=hh_SOC
        self.batt_size=batt_size
        self.E=batt_size
        self.Pbmax=batt_size*Crate
        self.battery=self.battery_management()
        self.Pb=self.battery[0]
        self.SOC2=self.battery[1]
        self.Pcurtail=self.battery[2]
        
    def battery_management(self):
        if self.hh_gen>self.hh_demand:
            self.Pbcal=(SOC_max-self.hh_SOC)*self.E/(nc*dT) #Charging
            if self.Pbcal>=self.Pbmax:
                self.Pbreq=self.Pbmax
            else:
                self.Pbreq=self.Pbcal
            if (self.hh_gen-self.hh_demand)>=self.Pbreq:
                self.Pb=self.Pbreq 
            else:
                self.Pb=(self.hh_gen-self.hh_demand)
        else:
            self.Pbcal=(self.hh_SOC-SOC_min)*(nd*self.E)/dT
            if self.Pbcal>=self.Pbmax:
                self.Pbav=self.Pbmax
            else:
                self.Pbav=self.Pbcal
            if (self.hh_demand-self.hh_gen)>=self.Pbav:
                self.Pb=-self.Pbav
            else:
                self.Pb=-(self.hh_demand-self.hh_gen) #discharge is -ve Pb
        self.Pcurtail=self.hh_demand-self.hh_gen+self.Pb
        if self.batt_size>0:
            self.SOCend=self.hh_SOC + ((self.Pb*nc)*(dT/self.E) if self.Pb>=0 else (self.Pb/nd)*(dT/self.E))
        else:
            self.SOCend=0
        return self.Pb, self.SOCend, self.Pcurtail

In [4]:
class LCOEoff:
    def __init__(self,SystemCost):
        self.SystemCost=SystemCost
        self.lcoeoff=self.calculate_lcoeoff()
        
    def calculate_lcoeoff(self):
        self.lcoeoff=((self.SystemCost) + (Inv[k-1]*10/15 if k>=1 else 0) + (Inv[k-2]*5/15 if k>=2 else 0))/(Pd.iloc[h,:].sum()*15)
        return self.lcoeoff

##### Object to calculate hourly bill of individual customer

In [5]:
class Bill:
    def __init__(self,demand,tierrate):
        self.monthlydemand=demand
        self.tierrate=tierrate
        self.electricity_bill=self.calculate_electricity_bill()
 
    def calculate_electricity_bill(self):
        bill_h=[]
        for i in range(0,12):
            bill=(self.tierrate[0] if self.monthlydemand[i] <= E1 else self.tierrate[1] if self.monthlydemand[i] <= E2 else self.tierrate[2] if self.monthlydemand[i] <= E3 else self.tierrate[2]+(self.monthlydemand[i]-E3)*0.15) 

            bill_h.append(bill)
        return sum(bill_h)

In [6]:
class Systemsize:
    def __init__(self,total_energy_consumption):
        self.total_energy_consumption=total_energy_consumption
        self.maxlimitpv=1
        panelsize=0.4 #kW
        # PVCost=3064.56 #$/kW
        annual_om_cost_per_kw=OnM.iloc[k*5]
        BattCost=Batteryadvanced.iloc[k*5]
        PVCost=PVadvanced.iloc[k*5]
        # BattCost=654 #$/kW

        X2=self.maxlimitpv*self.total_energy_consumption/(solar_prod_annual)
        X=X2-(pvk[k-1,h] if k>=1 else 0)-(pvk[k-2,h] if k>=2 else 0)
        X1=math.ceil(X/panelsize) #number of 0.4 kW PV panels
        # print("AEC/AEP per kW = ", X, "Number of 0.4kW Panels = ", X1)

        #PV size options: 100%, 50%, 25%, 0
        PV1=X1*panelsize 
        PV2=math.floor(X1/2)*panelsize
        PV3= math.floor(X1/4)*panelsize
        PV4=0
        data_PV = {'PVSize': [PV1, PV2, PV3, PV4],
                  'PVCost': [PVCost*PV1, PVCost*PV2, PVCost*PV3, PVCost*PV4]}
        df_PV = pd.DataFrame(data_PV)
        
        #Battery size options: round down values, closest to selected PV sizes
        B1=math.floor(PV1)/Crate
        B2=math.floor(PV2)/Crate
        B3=math.floor(PV3)/Crate
        B4=math.floor(PV4)/Crate
        # print('1:', 'PV=', PV1, "Batt=", B1, '2:', 'PV=', PV2, "Batt=",  B2, '3:', 'PV=', PV3, "Batt=", B3, '4:', 'PV=', PV4, "Batt=", B4)
        data_B = {'BattSize': [B1, B2, B3, B4],
                  'BattCost': [BattCost*B1, BattCost*B2, BattCost*B3, 0]} #price of battery
        df_B = pd.DataFrame(data_B)  
       
        self.df_combinations = pd.DataFrame(columns=['PVSize', 'BattSize', 'Total_Cost']) 

        for _, row_PV in df_PV.iterrows():
            for _, row_B in df_B.iterrows():
                total_cost = row_PV['PVCost'] + row_B['BattCost'] + annual_om_cost_per_kw*row_PV['PVSize']*systemlifetime
                data = [[row_PV['PVSize'], row_B['BattSize'], total_cost]]
                self.df_combinations = pd.concat([self.df_combinations, pd.DataFrame(data, columns=['PVSize', 'BattSize', 'Total_Cost'])])

        self.df_combinations.reset_index(drop=True, inplace=True)  # Reset the index
        print(self.df_combinations)

##### Object for individual customer's decision

In [7]:
class Decision:
    # def __init__(self,Electricity_Bill,PVSize,PVSize0,BattSize,BattSize0,BillPV0,SystemCost,BaseBilla,BaseLoada,Loff):
    def __init__(self,Electricity_Bill,PVSizedecision,BattSizedecision,SystemCost,Loff):
        self.electricity_bill=Electricity_Bill
        self.PVSizedecision=PVSizedecision
        self.BattSizedecision=BattSizedecision
        self.SystemCost=SystemCost
        self.options=options
        self.Loff=Loff
        self.LCOE=np.zeros(self.options)
        self.lcoge=self.calculate_lcoge() #levelized cost of grid electricity
        self.PV_lcoe=self.calculate_PV_lcoe() #levelized cost of each PV-batt combination
        self.system_decision=self.select_system() #logic to select one combination


    def calculate_lcoge(self): #previous investment?
        self.lcoge = round((Inv[k-2,h]*5/15 + Inv[k-1,h]*10/15 + Billexpected[k,h]*5 + Billkminus1k[h,15]*5 + Bill0[k,h]*5)/(Pd.iloc[h,:].sum()*15) if k>=2 else (Inv[k-1,h]*10/15 + Billexpected[k,h]*10 + Bill0[k,h]*5)/(Pd.iloc[h,:].sum()*15) if k==1 else (Bill0[k,h]*15)/(Pd.iloc[h,:].sum()*15),3)
        print('grid lcoe',self.lcoge)
        return self.lcoge
    
    def calculate_PV_lcoe(self):
        for i in range(0,self.options):
            self.LCOE[i]=round((Inv[k-2,h]*5/15+Inv[k-1,h]*10/15+EFCO*self.SystemCost[i]+self.electricity_bill[i]*5+Billkminus1k[h,i]*5+Billonlyk[h,i]*5)/(Pd.iloc[h,:].sum()*15) if (k-2) >= 0 else (Inv[k-1,h]*10/15+EFCO*self.SystemCost[i]+self.electricity_bill[i]*10+Billonlyk[h,i]*5)/(Pd.iloc[h,:].sum()*15) if k==1 else (EFCO*self.SystemCost[i] + self.electricity_bill[i]*15)/(Pd.iloc[h,:].sum()*15),3)
        return(self.LCOE)
    
    def select_system(self):
        print('LCOE combinations',self.LCOE)
        self.L= np.append(self.LCOE, self.Loff)
        self.PV_add_lcoe=min(filter(lambda y: y <= self.lcoge, self.L), default=None)
        index = np.where(self.L==self.PV_add_lcoe)[0]
        self.index = index[0]
        self.Selected_PV=self.PVSizedecision[self.index]
        self.SelectedBatt=self.BattSizedecision[self.index]

        self.Selected_Investment_Cost = self.SystemCost[self.index]

        print('selected',self.L[self.index])
        print('selected PV size', self.Selected_PV)
        print('selected Batt size', self.SelectedBatt)
        print('Selected_Investment_Cost', self.Selected_Investment_Cost)

        return self.PV_add_lcoe

##### Object for Retailer Cost Calculation- BaU

In [8]:
#Business-as-usual
#Object for retailer energy-related and fixed-costs
class Energy_costs: 
    def __init__(self,TotalLoad1,cont_vol,cont_price,WS_price,ret_margin_E,ret_margin_NE,risk_margin,admin_ch,fees,HourlyResLoad,hourlynet,hourly_load,hourly_ugen,connectedhouses,DVC,resnetload):
        self.cont_vol=cont_vol #contract volume
        self.cont_price=cont_price #contract price
        self.total_load=TotalLoad1
        self.ret_margin_E=ret_margin_E #energy return margin
        self.ret_margin_NE=ret_margin_NE #return margin on non-energy costs
        self.risk_margin=risk_margin
        self.hourlynetload=hourlynet
        self.hourly_load=hourly_load
        self.hourly_ugen=hourly_ugen
        self.no_of_HH=connectedhouses
        self.DVC=DVC
        self.resload=HourlyResLoad
        self.resnetload=resnetload

        self.energy_proc_cost=self.calculate_energy_proc_cost()
        self.Ret_Var_Cost_pu=self.calculate_Ret_Var_Cost_pu()
        self.ret_fixed_cost=self.calculate_ret_fixed_cost()
        self.ret_fixed_cost_HH=self.calculate_ret_fixed_cost_HH()
        # self.resepc=self.calculate_energyprocres()
        self.rate=self.calculate_rate()
        self.tierrate=self.calculate_tierrate()
        self.ret_rev=self.calculate_ret_rev()
        self.dsocharges=self.calculate_chargestoDSO()
       
    def calculate_energy_proc_cost(self): #energy procurment costs from contracts and WS markets
        epcost=(WS_price*np.maximum(0,(self.hourlynetload-cont_vol))+cont_vol*cont_price).sum(axis=1)
        print('market cost', (WS_price*np.maximum(0,(self.hourlynetload-cont_vol))).values.sum())
        print('cont cost', (cont_vol*cont_price).values.sum())          
        return epcost
    
    def calculate_Ret_Var_Cost_pu(self): #retailer's total variable costs
        print('EP cost', (self.energy_proc_cost.sum())*(1+self.ret_margin_E+self.risk_margin))
        print('load', (self.hourlynetload.values.sum()))
        rvc=((self.energy_proc_cost)*(1+self.ret_margin_E+self.risk_margin)/(self.hourly_load.sum(axis=1)))
        return rvc
    
    # def calculate_energyprocres(self):
    #     resepc=self.Ret_Var_Cost_pu*self.resload
    #     return resepc
    
    
    def calculate_ret_fixed_cost(self): #retailer's total fixed costs per hour over 15 years
        return (admin_ch)*(1+ret_margin_NE)
      
    def calculate_ret_fixed_cost_HH(self): #fixed costs per household: 
        return (self.ret_fixed_cost/(5*households.sum(axis=1)*8760)) #per hour charge
    
    def calculate_ret_rev(self):
        return ((self.Ret_Var_Cost_pu*base.sum(axis=1)*1000).sum())

    def calculate_rate(self):
        self.rates = ((self.ret_fixed_cost+Fixed_connection_charges)+(((self.Ret_Var_Cost_pu*self.resnetload.sum(axis=1))+(self.DVC*self.resload.sum(axis=1))).sum()))/(((x1*c1+x2*c2+x3*c3+x4*c4)*households).sum(axis=1))
        # self.rates = ((self.ret_fixed_cost+Fixed_connection_charges)+(((self.Ret_Var_Cost_pu+self.DVC)*(self.resload).sum(axis=1)).sum()))/(((x1*c1+x2*c2+x3*c3+x4*c4)*households).sum(axis=1))
        # self.rates = ((self.ret_fixed_cost+Fixed_connection_charges)+((self.resepc.sum(axis=1)+self.DVC*self.resload.sum(axis=1)).sum()))/(((x1*c1+x2*c2+x3*c3+x4*c4)*households).sum(axis=1))
        # print('total cost',(((self.Ret_Var_Cost_pu+self.DVC)*(self.resload).sum(axis=1)).sum()))
        print(self.rates, 'rate')
        return self.rates
    
    def calculate_tierrate(self):
        tierrate=[self.rate*x1/12, self.rate*x2/12, self.rate*x3/12, self.rate*x4/12]
        return tierrate
    
    def calculate_chargestoDSO(self):
        dsocharges=((self.DVC)*self.resload.sum(axis=1)).sum()+Fixed_connection_charges
        return dsocharges

##### Object for Delivery Cost Calculation

In [9]:
# class Delivery_Costs:
#     def __init__(self,total_load,h):
#         #margin included in these costs
#         self.Fixed_connection_charges=Fixed_connection_charges #fixed distribution connection charges
#         self.total_load=total_load
#         self.no_of_HH=h
#         self.T_cost=self.calculate_T_cost()
#         self.D_cost=self.calculate_D_cost()
#         self.M_cost=self.calculate_M_cost()
#         self.dist_var_cost=self.calculate_dist_var_cost()
#         self.dist_var_cost_pu=self.calculate_dist_var_cost_pu()
#         self.dist_fixed_cost=self.calculate_dist_fixed_cost()

#     def calculate_T_cost(self):
#         return Tcost/self.total_load
    
#     def calculate_D_cost(self):
#         return Dcost/self.total_load
    
#     def calculate_M_cost(self):
#         return Mcost/self.total_load
    
#     def calculate_dist_var_cost(self): #variable costs per kWh
#         print('Var Delivery Cost',(self.T_cost+self.D_cost+self.M_cost))
#         return (self.T_cost+self.D_cost+self.M_cost)
    
#     def calculate_dist_var_cost_pu(self): #variable costs per kWh
#         return (Tcost+Dcost+Mcost)/self.total_load

#     def calculate_dist_fixed_cost(self): #fixed costs per house
#         # print('Fixed Delivery Cost', self.Fixed_connection_charges)
#         return (self.Fixed_connection_charges)

In [10]:
class Delivery_Costs:
    def __init__(self,total_load,h):
        #margin included in these costs
        self.Fixed_connection_charges=Fixed_connection_charges #fixed distribution connection charges
        self.total_load=total_load
        self.no_of_HH=h
        self.T_cost=self.calculate_T_cost()
        self.D_cost=self.calculate_D_cost()
        self.M_cost=self.calculate_M_cost()
        self.dist_var_cost_pu=self.calculate_dist_var_cost_pu()
        self.dist_fixed_cost_HH=self.calculate_dist_fixed_cost_HH()

    def calculate_T_cost(self):
        return Tcost
    # /self.total_load
    
    def calculate_D_cost(self):
        return Dcost
    # /self.total_load
    
    def calculate_M_cost(self):
        return Mcost
    # /self.total_load
    
    def calculate_dist_var_cost_pu(self): #variable costs per kWh
        return (Tcost+Dcost+Mcost)
    # *self.total_load
    
    def calculate_dist_fixed_cost_HH(self): #fixed costs per house
        return (self.Fixed_connection_charges/(5*households.values.sum()*8760))

In [ ]:
EFCO=1

kmax=3
houses=5
totalhouses=321900.6
yy=0
yyend=15
cagr=0.016
cagrhouses=0.02
planninghorizon=yyend-yy
hours=8760
cont_price=0.15
ret_margin_E=0.15
ret_margin_NE=0.05
risk_margin=0.05
Tcost=0.042094
Dcost=0.013033
Mcost=0.017162
# Tcost=0.042094*5.5e+10*planninghorizon
# Dcost=0.013033*5.5e+10*planninghorizon
# Mcost=0.017162*5.5e+10*planninghorizon
Fixed_connection_charges=0.647903*365*planninghorizon*totalhouses*5
Other_ch=0
# 0.002252 #$/kWh balance pool allocation
rate_riders=0 #enter the values as $/h 
admin_ch=0.2392*totalhouses*5*365*planninghorizon
fees=0
SOC_min=0.10
SOC_max=1
Crate=1
Pr_th_s=0.16
Pr_th_b=0.14
dT=1
nc=0.95
nd=0.95

x1=1
x2=1.6
x3=2.35
x4=3.1

E1=300
E2=600
E3=900

systemlifetime=15 #system lifetime in years
solar_prod_annual=1167   #kWh/yr
OnM=pd.read_csv('o&mcost.csv', delimiter=',', header=None)
Pg = pd.read_csv('PV.csv', delimiter=',', header=None)
contractsize = pd.read_csv('cont_vol.csv', delimiter=',', header=None)
WSprice = pd.read_csv('WSprice.csv',usecols=[1], delimiter=',', header=None)
S2 = pd.read_csv('soc.csv', delimiter=',', header=None) #5000 houses
Pd = pd.read_csv('house_load_selected.csv',delimiter=',', header=None) #5 houses
Batteryadvanced= pd.read_csv('battcost1.csv', delimiter=',', header=None)
PVadvanced=pd.read_csv('pvcost1.csv', delimiter=',', header=None)
baseload = pd.read_csv('baseload.csv',delimiter=',', header=None) #1 year

In [12]:
Retailer_VC=np.zeros((kmax+1,hours))
Retailer_FC=np.zeros(kmax)
T_cost=np.zeros(kmax)
D_cost=np.zeros(kmax)
M_cost=np.zeros(kmax)
D_Fcost=np.zeros(kmax)

DSO_charges=np.zeros((kmax))
DSO_charges_actual=np.zeros((kmax))
Retailer_VC_actual=np.zeros((kmax+1,hours))
Retailer_FC_actual=np.zeros(kmax)
Retailer_Rev=np.zeros((kmax))
Retailer_Rev_actual=np.zeros((kmax))
Retailer_Revenue_Household=np.zeros((kmax))
Retailer_Revenue_Household_actual=np.zeros((kmax))
profit=np.zeros(kmax)
actualprofit=np.zeros(kmax)

energycost=np.zeros(kmax)
energycostactual=np.zeros(kmax)

T_cost_actual=np.zeros(kmax)
D_cost_actual=np.zeros(kmax)
M_cost_actual=np.zeros(kmax)
D_Fcost_actual=np.zeros(kmax)
st=np.zeros(houses)

ugenpayment=np.zeros(kmax)
ugenpaymentactual=np.zeros(kmax)
Electricity_Bill=np.zeros((16,houses))
Electricity_Bill_actual=np.zeros((kmax,houses))
Actualbill=np.zeros((kmax,houses))
Actualbillkminus1k=np.zeros((kmax,houses))
Actualbillonlyk=np.zeros((kmax,houses))
Actualbill0=np.zeros((kmax,houses))
Actualdcharge=np.zeros((kmax,houses))

Bill0=np.zeros((kmax,houses))

Credits=np.zeros((kmax,houses))
Credits_actual=np.zeros((kmax,houses))
Demand=np.zeros((houses,hours))
Base_demand=np.zeros((hours))
EC2=0
EG2=0
EB2=0
hh=0
system_size_b = pd.DataFrame(0, index=range(houses), columns=[""])
batt=np.zeros(houses)

battk=np.zeros((kmax,houses))
pvk=np.zeros((kmax,houses))

s=pd.DataFrame(kmax, index=range(houses), columns=[""])
Inv=np.zeros((kmax,houses))
flags = [False] * houses

Billexpected=np.zeros((kmax,houses))
Planrate=np.zeros((kmax))
Actualplanrate=np.zeros((kmax))

In [13]:
for k in range(0,kmax):
    print('k=', k)
    sumdf=pd.DataFrame()
    sumdf0=pd.DataFrame()
    sumdf1=pd.DataFrame()
    sumdf2=pd.DataFrame()
    sumdf3=pd.DataFrame()
    sumdf1actual=pd.DataFrame()
    sumdf2actual=pd.DataFrame()
    sumdf3actual=pd.DataFrame()

    base=pd.DataFrame()
    Phour=pd.DataFrame()
    Pinj=pd.DataFrame()
    cont_vol = pd.DataFrame()
    WS_price = pd.DataFrame()
    households = pd.DataFrame()
    Phour2=pd.DataFrame()
    Pinj2=pd.DataFrame()
    c1=0
    c2=0
    c3=0
    c4=0
    c11=0
    c21=0
    c31=0
    c41=0
    
    
    for y in range(yy,yyend):
        ind=y-yy
        base[ind] = baseload*(1+cagr)**(y)
    
    for y in range(yy,yyend):
        ind=y-yy
        households[ind] = [totalhouses*(1+cagrhouses)**(y)]
    print(sum(households))    
    
    WS_price1=(WSprice[(0*hours):(hours*15)]/1000)
    WS_price = pd.DataFrame()
    values_per_column = 8760
    num_columns = 15
    for i in range(num_columns):
        start_idx = i * values_per_column
        end_idx = (i + 1) * values_per_column
        WS_price[i] = WS_price1[start_idx:end_idx].reset_index(drop=True)
         
    lcoeoffgrid = np.zeros(houses)
    PV = np.zeros(houses)
    Battery = np.zeros(houses)
    
    CustomerLoad0 = pd.DataFrame()
    CustomerLoad1 = pd.DataFrame()
    CustomerBatt1 = pd.DataFrame()
    CustomerSOC1 = pd.DataFrame()
    Monthlyavgcons = np.zeros(houses)
    totalcons = np.zeros(houses)
    totalcons1 = np.zeros(houses)
    
    Customerloadactual = np.zeros((hours,houses))
    Customersocactual = np.zeros((hours+1,houses))
    Customerbattactual = np.zeros((hours,houses))
    CustomerLoadPVonlykactual = np.zeros((hours,houses))
    Loadkminus1kactual = np.zeros((hours,houses))

    ECurtail = np.zeros((houses,hours))
    CustomerSOCOff = np.zeros((houses,hours+1))
    CustomerbattOff = np.zeros((houses,hours))

    BillPVCombinations = np.zeros((houses,16))
    Billonlyk = np.zeros((houses,16))
    Billkminus1k = np.zeros((houses,16))
   
    Dcharge = np.zeros((houses,16))
    Echarge = np.zeros((houses,16))
    
    #EM with initial PV and Battery (Bill(k-1,k-2)) to get historic load estimate for retailer and DSO
    for h in range(0,houses):
        loads1 = []
        batp1 = []
        soc1 = []
        PV[h]=(pvk[k-2,h] if k>=2 else 0) + (pvk[k-1,h] if k>=1 else 0)
        Battery[h]=(battk[k-2,h] if k>=2 else 0) + (battk[k-1,h] if k>=1 else 0)
        SOC=SOC_max if Battery[h]>0 else 0
        soc1.append(SOC)
        for t in range(0,hours): #8760
            instance1 = Customer(Pd.iloc[h,t], PV[h], SOC, Battery[h], Retailer_VC[k,t], WS_price.iloc[t,8])
            SOC=instance1.SOC2
            loads1.append(instance1.Pgrid)
            batp1.append(instance1.Pb)
            soc1.append(instance1.SOC2)
        column_name = f'House {h}'
        
        CustomerLoad1[column_name] = loads1
        CustomerBatt1[column_name] = batp1
        data=np.array(loads1)
        data_reshaped = data.reshape(12, 730)
        sum_per_month = np.sum(data_reshaped, axis=1)
        # for month, total in enumerate(sum_per_month, start=1):
        #     print(f"Month {month}: {total}")
        sumdf[f'House{h}'] = sum_per_month
        # print(sumdf[f'House{h}'])
        
#         for i in range(0,12):
#             c1=[]
#             c2=[]
#             c3=[]
#             c4=[]
#             c1=[x for x in list(sumdf[f'House{h}',i]) if x <=E1]
#             c2=[x for x in list(sumdf[f'House{h}',i]) if E1 < x <= E2]
#             c3=[x for x in list(sumdf[f'House{h}',i]) if E2 < x <= E3]
#             c4=[x for x in list(sumdf[f'House{h}',i]) if x > E3]
#             count[i]=[len(c1), len(c2), len(c3), len(c4)]

        # Monthlyavgcons[h] = (np.sum(loads1)/12)                
        # c1=c1+[1 if Monthlyavgcons[h] <= E1 else 0][0]
        # c2=c2+[1 if E1 < Monthlyavgcons[h] <= E2 else 0][0]
        # c3=c3+[1 if E2 < Monthlyavgcons[h] <= E3 else 0][0]
        # c4=c4+[1 if Monthlyavgcons[h] > E3 else 0][0]
        E11=3600
        E21=7200
        E31=10800
        totalcons[h] = (np.sum(loads1))                
        c1=c1+[1 if totalcons[h] <= E11 else 0][0]
        c2=c2+[1 if E11 < totalcons[h] <= E21 else 0][0]
        c3=c3+[1 if E21 < totalcons[h] <= E31 else 0][0]
        c4=c4+[1 if totalcons[h] > E31 else 0][0]
    print(c1,c2,c3,c4)

    Load1 = np.where(CustomerLoad1 >= 0, CustomerLoad1, 0)
    Inj1 = np.where(CustomerLoad1 < 0, CustomerLoad1, 0)
    TotalLoad_h = Load1.sum() #kWh
    TotalInj_h = Inj1.sum() #kWh
    HourlyLoad1_h = (Load1.sum(axis=1)) #kWh
    HourlyInj1_h = Inj1.sum(axis=1)
    HourlyLoad0 = pd.DataFrame(HourlyLoad1_h)
    HourlyInj0 = pd.DataFrame(HourlyInj1_h)
    # HourlyLoad1 = (HourlyLoad0*321900.6)+(base*1000) #kWh
    # HourlyInj1 = HourlyInj0*321900.6
    # cont_vol=contractsize[0]*1000

    for y in range(yy,yyend):
        ind = y-yy
        cont_vol[ind] = contractsize[0]*1000 #kWh

    for y in range(yy,yyend):
        ind = y-yy
        Phour[ind] = HourlyLoad0[0]*households.iloc[0,ind]
    HourlyLoad1=(Phour)+(base*1000) #kWh
    HourlyResLoad1 = (Phour) #kWh

    for y in range(yy,yyend):
        ind = y-yy
        Pinj[ind] = HourlyInj0[0]*households.iloc[0,ind]
    HourlyInj1 = (Pinj) #kWh
    
    HourlyNet1=HourlyLoad1+HourlyInj1
    NetResLoad1=HourlyResLoad1+HourlyInj1

    TotalLoad1 = (TotalLoad_h*households.values.sum())+(base.values.sum()*1000) #kWh
    print('TotalLoad_h',TotalLoad_h)
    Totalresload1=(TotalLoad_h*households.values.sum()) #kWh
    TotalInj1=(TotalInj_h*households.values.sum()) #kWh
    
    TotalNet1=TotalLoad1+TotalInj1
    
    print('Total Grid Load for 15 years:', TotalLoad1, 'kWh')
    print('Total Res Load for 15 years:', Totalresload1, 'kWh')
    print('Total Injection for 15 years:', TotalInj1, 'kWh')
    
    # Distribution=Delivery_Costs(TotalLoad1,houses-hh)
    Distribution=Delivery_Costs(Totalresload1,houses-hh)
    Retailer = Energy_costs(TotalNet1,cont_vol,cont_price,WS_price,ret_margin_E,ret_margin_NE,risk_margin,admin_ch,fees,HourlyResLoad1,HourlyNet1,HourlyLoad1,HourlyInj1,houses-hh,Distribution.dist_var_cost_pu,NetResLoad1)
    Planrate[k]=Retailer.rate

    energycost[k]=Retailer.energy_proc_cost.sum()
    # Retailer_VC[k]=Retailer.Ret_Var_Cost_pu
    # Retailer_FC[k]=Retailer.ret_fixed_cost_HH
    Retailer_Rev[k]=Retailer.ret_rev
    DSO_charges[k]=Retailer.dsocharges
    print('Total Energy Cost',energycost[k] ,'C$')
    
    for h in range(0,houses):
        billexp = Bill(sumdf[f'House{h}'], Retailer.tierrate)
        Billexpected[k,h] = billexp.electricity_bill
    Retailer_Revenue_Household[k]=Billexpected[k].sum()*(households.sum(axis=1))
    profit[k]=Retailer_Revenue_Household[k]+Retailer_Rev[k]-energycost[k]-DSO_charges[k]-admin_ch
    print("Retailer revenue total", Retailer_Revenue_Household[k]+Retailer_Rev[k], 'C$')
    print('Cost to DSO', DSO_charges[k], 'C$')
    print('Retailer Profit', profit[k], 'C$')

    for h in range(0,houses):
        loads0 = []
        SOC0=0
        batt0=0
        for t in range(0,hours):
            Generation0=0
            instance0 = Customer(Pd.iloc[h,t],Generation0, SOC0, batt0, Retailer_VC[k,t], WS_price.iloc[t,8])
            loads0.append(instance0.Pgrid)
        column_name = f'House {h}'
        CustomerLoad0[column_name] = loads0
        data=np.array(loads0)
        data_reshaped = data.reshape(12, 730)
        sum_per_month = np.sum(data_reshaped, axis=1)
        sumdf0[f'House{h}'] = sum_per_month

        bill0 = Bill(sumdf0[f'House{h}'], Retailer.tierrate)
        Bill0[k,h] = bill0.electricity_bill

        systemsize=Systemsize(Pd.iloc[h,:].sum())
        options=len(systemsize.df_combinations)
        
        #OFFGRID OPERATION
        SOC=SOC_max #fully charged battery
        PVSizeOff=systemsize.df_combinations.iloc[0,0]+PV[h]
        BattSizeOff=systemsize.df_combinations.iloc[0,1]+Battery[h]
        SystemCostOff=systemsize.df_combinations.iloc[0,2]
        genoff=[]
        curtailoff=[]
        battoff=[]
        CustomerSOCOff[h,0]=SOC_max if BattSizeOff>0 else 0
        SOC=CustomerSOCOff[h,0]
        for t in range(0,hours):
            instanceOff = CustomerOffgrid(Pd.iloc[h,t], PVSizeOff, SOC, BattSizeOff)
            SOC=instanceOff.SOC2 
            CustomerSOCOff[h,t+1]=SOC
            ECurtail[h,t]=instanceOff.Pcurtail
            CustomerbattOff[h,t]=instanceOff.Pb 

        if all(x <= 0 for x in ECurtail[h]):
            loff=LCOEoff(SystemCostOff)
            lcoeoffgrid[h]=loff.lcoeoff
        else:
            lcoeoffgrid[h]=100  
 
        # PV-Batt Combinations
        #ONLY new options at k, no previous PV-Batt
        if flags[h]==False:
            CustomerLoadPVonlyk=np.zeros((hours,options))
            CustomerSOCPVonlyk=np.zeros((hours+1,options))
            CustomerBattPVonlyk=np.zeros((hours,options))
            PVSizeonlyk=np.zeros(options)
            BattSizeonlyk=np.zeros(options)
            SystemCost=np.zeros(options)
            for i in range (0,options):
                PVSizeonlyk[i]=systemsize.df_combinations.iloc[i,0]
                BattSizeonlyk[i]=systemsize.df_combinations.iloc[i,1]
                SystemCost[i]=systemsize.df_combinations.iloc[i,2]
                load3=[]
                gen3=[]
                soc3=[]
                batt3=[]
                CustomerSOCPVonlyk[0,i]=SOC_max if BattSizeonlyk[i]>0 else 0
                SOC=CustomerSOCPVonlyk[0,i]
                for t in range(0,hours):
                    instance3 = Customer(Pd.iloc[h,t], PVSizeonlyk[i], SOC, BattSizeonlyk[i], Retailer_VC[k,t], WS_price.iloc[t,8])
                    SOC=instance3.SOC2
                    CustomerLoadPVonlyk[t,i]=instance3.Pgrid
                    CustomerSOCPVonlyk[t+1,i]=instance3.SOC2
                    CustomerBattPVonlyk[t,i]=instance3.Pb
                data=np.array(CustomerLoadPVonlyk[:,i])
                data_reshaped = data.reshape(12, 730)
                sum_per_month = np.sum(data_reshaped, axis=1)
                sumdf1[f'House{h}',i] = sum_per_month

                billonlyk = Bill(sumdf1[f'House{h}',i], Retailer.tierrate)
                Billonlyk[h,i]=billonlyk.electricity_bill

            # PREVIOUS PV-BATT at k-1 and new options at k
            Loadkminus1k=np.zeros((hours,options))
            SOCkminus1k=np.zeros((hours+1,options))
            Battkminus1k=np.zeros((hours,options))
            PVSizekminus1k=np.zeros(options)
            BattSizekminus1k=np.zeros(options)
            for i in range (0,options):
                PVSizekminus1k[i]=(systemsize.df_combinations.iloc[i,0]) + (pvk[k-1,h] if k>=1 else 0)
                BattSizekminus1k[i]=(systemsize.df_combinations.iloc[i,1]) + (battk[k-1,h] if k>=1 else 0)
                load4=[]
                gen4=[]
                soc4=[]
                batt4=[]
                SOCkminus1k[0,i]=SOC_max if BattSizekminus1k[i]>0 else 0
                SOC=SOCkminus1k[0,i]
                for t in range(0,hours):
                    instance4 = Customer(Pd.iloc[h,t], PVSizekminus1k[i], SOC, BattSizekminus1k[i], Retailer_VC[k,t], WS_price.iloc[t,8])
                    SOC=instance4.SOC2
                    Loadkminus1k[t,i]=instance4.Pgrid
                    SOCkminus1k[t+1,i]=instance4.SOC2
                    Battkminus1k[t,i]=instance4.Pb
                data=np.array(Loadkminus1k[:,i])
                data_reshaped = data.reshape(12, 730)
                sum_per_month = np.sum(data_reshaped, axis=1)
                sumdf2[f'House{h}',i] = sum_per_month

                billkminus1k = Bill(sumdf2[f'House{h}',i], Retailer.tierrate)
                Billkminus1k[h,i]=billkminus1k.electricity_bill

            # PREVIOUS PV-BATT at k-1, k-2, and new options at k
            CustomerLoadPV=np.zeros((hours,options))
            CustomerSOCPV=np.zeros((hours+1,options))
            CustomerBattPV=np.zeros((hours,options))
            PVSize=np.zeros(options)
            BattSize=np.zeros(options)
            for i in range (0,options):
                PVSize[i]=systemsize.df_combinations.iloc[i,0]+PV[h]
                BattSize[i]=systemsize.df_combinations.iloc[i,1]+Battery[h]
                load2=[]
                gen2=[]
                soc2=[]
                batt2=[]
                CustomerSOCPV[0,i]=SOC_max if BattSize[i]>0 else 0
                SOC=CustomerSOCPV[0,i]
                for t in range(0,hours):
                    instance2 = Customer(Pd.iloc[h,t], PVSize[i], SOC, BattSize[i], Retailer_VC[k,t], WS_price.iloc[t,8])
                    SOC=instance2.SOC2
                    CustomerLoadPV[t,i]=instance2.Pgrid
                    CustomerSOCPV[t+1,i]=instance2.SOC2
                    CustomerBattPV[t,i]=instance2.Pb
                data=np.array(CustomerLoadPV[:,i])
                data_reshaped = data.reshape(12, 730)
                sum_per_month = np.sum(data_reshaped, axis=1)
                sumdf3[f'House{h}',i] = sum_per_month

                bill = Bill(sumdf3[f'House{h}',i], Retailer.tierrate)
                BillPVCombinations[h,i]=bill.electricity_bill
                # Echarge[h,i]=bill.electricity_charges
                # Dcharge[h,i]=bill.delivery_charges

            PVSizedecision=np.append(systemsize.df_combinations.iloc[:,0].values,PVSizeOff)
            BattSizedecision=np.append(systemsize.df_combinations.iloc[:,1], BattSizeOff)
            SystemCost=np.append(SystemCost,SystemCostOff)

            decision=Decision(BillPVCombinations[h],PVSizedecision,BattSizedecision,SystemCost,lcoeoffgrid[h]) #change name to lcoe

            ind=decision.index
            battk[k,h]=decision.SelectedBatt
            pvk[k,h]=decision.Selected_PV
            Inv[k,h]=SystemCost[ind]

            if ind==options:
                hh=hh+1
                flags[h] = True  # Set the flag to True for the houses that go off-grid
                Customerloadactual[:,h]==np.zeros(hours)
            else:
                Customerloadactual[:,h]=CustomerLoadPV[:,ind]
                totalcons1[h] = (np.sum(Customerloadactual[:,h]))
                c11=c11+[1 if totalcons1[h] <= E11 else 0][0]
                c21=c21+[1 if E11 < totalcons1[h] <= E21 else 0][0]
                c31=c31+[1 if E21 < totalcons1[h] <= E31 else 0][0]
                c41=c41+[1 if totalcons1[h] > E31 else 0][0]
                print(c11,c21,c31,c41)
            sumdf3actual[f'House{h}']=sumdf3[f'House{h}',ind]
            sumdf2actual[f'House{h}']=sumdf2[f'House{h}',ind]
            sumdf1actual[f'House{h}']=sumdf1[f'House{h}',ind]
            

            # Customerbattactual[:,h]=CustomerBattPV[:,ind]
            # Customersocactual[:,h]=CustomerSOCPV[:,ind]
            # Loadkminus1kactual[:,h]=Loadkminus1k[:,ind]
            # CustomerLoadPVonlykactual[:,h]=CustomerLoadPVonlyk[:,ind]
            # Billexpected[k,h]=Billkminus1k[h,ind]

    Load2 = np.where(Customerloadactual >= 0, Customerloadactual, 0)
    Inj2 = np.where(Customerloadactual < 0, Customerloadactual, 0)
    TotalLoad2_h = Load2.sum() #kWh
    TotalInj2_h = Inj2.sum() #kWh
    HourlyLoad2_h = (Load2.sum(axis=1)) #kWh
    HourlyInj2_h = (Inj2.sum(axis=1)) #kWh
    HourlyLoad20 = pd.DataFrame(HourlyLoad2_h)
    HourlyInj20 = pd.DataFrame(HourlyInj2_h)
    
    for y in range(yy,yyend):
        ind=y-yy
        Phour2[ind] = HourlyLoad20[0]*households.iloc[0,ind]
    HourlyLoad2=(Phour2)+(base*1000) #kWh
    HourlyResLoad2=Phour2

    for y in range(yy,yyend):
        ind=y-yy
        Pinj2[ind] = HourlyInj20[0]*households.iloc[0,ind]
    HourlyInj2 = (Pinj2) #kWh
    
    HourlyNet2=HourlyLoad2+HourlyInj2
    NetResLoad2=HourlyResLoad2+HourlyInj2
    
    TotalLoad2=(TotalLoad2_h*households.values.sum())+(base.values.sum()*1000) #kWh
    Totalresload2=(TotalLoad2_h*households.values.sum()) #kWh
    TotalInj2=(TotalInj2_h*households.values.sum()) #kWh
    
    TotalNet2=TotalLoad2+TotalInj2
    
    print('Actual Annual Load:',TotalLoad2, 'kWh')
    print('Actual Res Load:',Totalresload2, 'kWh')
    print('Actual Annual Injection:', TotalInj2, 'kWh')
    
    # Distributionactual=Delivery_Costs(TotalLoad2,houses-hh)
    Distributionactual=Delivery_Costs(Totalresload2,houses-hh)
    Retaileractual=Energy_costs(TotalNet2,cont_vol,cont_price,WS_price,ret_margin_E,ret_margin_NE,risk_margin,admin_ch,fees,HourlyResLoad2,HourlyNet2,HourlyLoad2,HourlyInj2,houses-hh,Distributionactual.dist_var_cost_pu,NetResLoad2)
    Actualplanrate[k]=Retaileractual.rate
    energycostactual[k]=Retaileractual.energy_proc_cost.sum()
    # Retailer_VC_actual[k]=Retaileractual.Ret_Var_Cost_pu
    # Retailer_FC_actual[k]=Retaileractual.ret_fixed_cost_HH
    Retailer_Rev_actual[k]=Retaileractual.ret_rev
    DSO_charges_actual[k]=Retaileractual.dsocharges
    # ugenpaymentactual[k]=Retaileractual.ret_ugen_payment
    # actualprofit[k]=Retailer_Rev_actual[k]-energycostactual[k]
    Retailer_VC[k+1]=Retaileractual.Ret_Var_Cost_pu
    # T_cost_actual[k]=Distributionactual.T_cost
    # D_cost_actual[k]=Distributionactual.D_cost
    # M_cost_actual[k]=Distributionactual.M_cost
    # D_Fcost_actual[k]=Distributionactual.dist_fixed_cost_HH
    print('Actual Total Energy Cost',energycostactual[k] ,'C$')
    # print("Actual Retailer VC", Retailer_VC_actual[k], 'C$/kWh')
    # print("Actual Retailer FC", Retailer_FC_actual[k], 'C$/h')
    # print("Actual Transmission cost", T_cost_actual[k], 'C$/kWh')
    # print("Actual Distribution cost", D_cost_actual[k], 'C$/kWh')
    # print("Actual Municipal cost", M_cost_actual[k], 'C$/kWh')
    # print('Actual Retailer Revenue', Retailer_Rev_actual[k], 'C$')
    
# #     fig = go.Figure()
# #     fig.add_traces(go.Scatter(y = HourlyLoad2, mode = 'lines', line=dict(color=colors[0]), name='Load'))
# #     fig.add_traces(go.Scatter(y = Retailer_VC_actual[k], mode = 'lines', line=dict(color=colors[1]), name='Retail Price'))
# #     fig.update_layout(title='Actual')
    
# #         # pio.write_html(fig, 'Offgrid.html')
    
# BILL WITH investment at k-2,k-1,k
    for h in range(0,houses):
        if flags[h]==False:
            bill_actual=Bill(sumdf3actual[f'House{h}'], Retaileractual.tierrate)
            Actualbill[k,h]=bill_actual.electricity_bill #total bill of each house

            bill_actual_kminus1k=Bill(sumdf2actual[f'House{h}'], Retaileractual.tierrate)
            Actualbillkminus1k[k,h]=bill_actual_kminus1k.electricity_bill

            bill_actual_onlyk=Bill(sumdf1actual[f'House{h}'], Retaileractual.tierrate)
            Actualbillonlyk[k,h]=bill_actual_onlyk.electricity_bill       

            bill0actual = Bill(sumdf0[f'House{h}'], Retaileractual.tierrate)
            Actualbill0[k,h] = bill0actual.electricity_bill
    print(sumdf3actual) 
    Retailer_Revenue_Household_actual[k]=Actualbill[k].sum()*(households.sum(axis=1))
    actualprofit[k]=Retailer_Revenue_Household_actual[k]+Retailer_Rev_actual[k]-energycostactual[k]-DSO_charges_actual[k]-admin_ch
    print("Retailer revenue total", Retailer_Revenue_Household_actual[k]+Retailer_Rev_actual[k], 'C$')
    print('Cost to DSO', DSO_charges_actual[k], 'C$')
    print('Retailer Profit', actualprofit[k], 'C$')
    
    dfpv = pd.DataFrame({f'PV{j+1}': pvk[:, j] for j in range(pvk.shape[1])})
    dfb = pd.DataFrame({f'Batt{j+1}': battk[:, j] for j in range(battk.shape[1])})
    print(dfpv)
    print(dfb)
    print("-------------------------------")
    yy=yy+5
    yyend=yyend+5

k= 0
105
1 1 2 1
TotalLoad_h [37714.60083216]
Total Grid Load for 15 years: [8.79150465e+11] kWh
Total Res Load for 15 years: [2.0994818e+11] kWh
Total Injection for 15 years: 0.0 kWh
market cost [1.7442721e+10]
cont cost 93543179220.0
EP cost [1.3318308e+11]
load [8.79150465e+11]
0    955.016696
dtype: float64 rate


C:\Users\ASUS\AppData\Local\Temp\ipykernel_52416\3442198059.py:174: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  Planrate[k]=Retailer.rate
C:\Users\ASUS\AppData\Local\Temp\ipykernel_52416\3442198059.py:185: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  Billexpected[k,h] = billexp.electricity_bill
C:\Users\ASUS\AppData\Local\Temp\ipykernel_52416\3442198059.py:186: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  Retailer_Revenue_Household[k]=Billexpected[k].sum()*(households.sum(axis=1))


Total Energy Cost 110985900203.14035 C$
Retailer revenue total 152894964869.30594 C$
Cost to DSO 20886283935.679928 C$
Retailer Profit 18914943411.62566 C$


C:\Users\ASUS\AppData\Local\Temp\ipykernel_52416\3442198059.py:208: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  Bill0[k,h] = bill0.electricity_bill


    PVSize  BattSize                                 Total_Cost
0      3.2       3.0  0    13482.603438
Name: 0, dtype: float64
1      3.2       1.0  0    12238.480308
Name: 0, dtype: float64
2      3.2       0.0  0    11616.418743
Name: 0, dtype: float64
3      3.2       0.0  0    11616.418743
Name: 0, dtype: float64
4      1.6       3.0   0    7674.394066
Name: 0, dtype: float64
5      1.6       1.0   0    6430.270937
Name: 0, dtype: float64
6      1.6       0.0   0    5808.209372
Name: 0, dtype: float64
7      1.6       0.0   0    5808.209372
Name: 0, dtype: float64
8      0.8       3.0    0    4770.28938
Name: 0, dtype: float64
9      0.8       1.0   0    3526.166251
Name: 0, dtype: float64
10     0.8       0.0   0    2904.104686
Name: 0, dtype: float64
11     0.8       0.0   0    2904.104686
Name: 0, dtype: float64
12     0.0       3.0   0    1866.184695
Name: 0, dtype: float64
13     0.0       1.0    0    622.061565
Name: 0, dtype: float64
14     0.0       0.0           0    0.0


...

C:\Users\ASUS\AppData\Local\Temp\ipykernel_52416\3442198059.py:208: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  Bill0[k,h] = bill0.electricity_bill


    PVSize  BattSize                                 Total_Cost
0      2.0       2.0  0    4062.712184
Name: 10, dtype: float64
1      2.0       0.0  0    3200.526022
Name: 10, dtype: float64
2      2.0       0.0  0    3200.526022
Name: 10, dtype: float64
3      2.0       0.0  0    3200.526022
Name: 10, dtype: float64
4      0.8       2.0   0    2142.39657
Name: 10, dtype: float64
5      0.8       0.0  0    1280.210409
Name: 10, dtype: float64
6      0.8       0.0  0    1280.210409
Name: 10, dtype: float64
7      0.8       0.0  0    1280.210409
Name: 10, dtype: float64
8      0.4       2.0  0    1502.291366
Name: 10, dtype: float64
9      0.4       0.0   0    640.105204
Name: 10, dtype: float64
10     0.4       0.0   0    640.105204
Name: 10, dtype: float64
11     0.4       0.0   0    640.105204
Name: 10, dtype: float64
12     0.0       2.0   0    862.186161
Name: 10, dtype: float64
13     0.0       0.0          0    0.0
Name: 10, dtype: float64
14     0.0       0.0          0    0.0
N

C:\Users\ASUS\AppData\Local\Temp\ipykernel_52416\3442198059.py:248: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  SystemCost[i]=systemsize.df_combinations.iloc[i,2]
C:\Users\ASUS\AppData\Local\Temp\ipykernel_52416\3442198059.py:267: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  Billonlyk[h,i]=billonlyk.electricity_bill
C:\Users\ASUS\AppData\Local\Temp\ipykernel_52416\3442198059.py:248: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  SystemCost[i]=systemsize.df_combinations.iloc[i,2]
C:\Users\ASUS\AppData\Local\Temp\ipykernel_52416\3442198059.py:267: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  Bill

grid lcoe 0.268
LCOE combinations [0.265 0.254 0.254 0.254 0.276 0.265 0.265 0.265 0.273 0.262 0.262 0.262
 0.279 0.268 0.268 0.268]
selected 0.254
selected PV size 2.0
selected Batt size 0.0
Selected_Investment_Cost 3200.5260224
3 2 0 0
Actual Annual Load: 947505177234.8528 kWh
Actual Res Load: 163183000022.10504 kWh
Actual Annual Injection: -67623366558.59596 kWh
market cost 16595937664.941902
cont cost 93543179220.0
EP cost 132166940261.93028
load 879881810676.2568
0    714.637462
dtype: float64 rate
Actual Total Energy Cost 110139116884.94191 C$
        House0      House1      House2       House3      House4
0   300.449455  749.261720  859.757828  1049.708298  264.482118
1   258.703267  634.590131  785.132931   953.095475  215.367698
2   166.718534  287.885142  514.475340   675.485233   10.993153
3   142.358293  226.697034  368.162044   507.883339  -19.046593
4   157.300229  262.811160  198.936008   310.802707  -32.797490
5   134.065528  212.868836   -9.112550   253.535836  -31.845

C:\Users\ASUS\AppData\Local\Temp\ipykernel_52416\3442198059.py:325: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  BillPVCombinations[h,i]=bill.electricity_bill
C:\Users\ASUS\AppData\Local\Temp\ipykernel_52416\3442198059.py:399: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  Actualplanrate[k]=Retaileractual.rate
C:\Users\ASUS\AppData\Local\Temp\ipykernel_52416\3442198059.py:431: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  Actualbill[k,h]=bill_actual.electricity_bill #total bill of each house
C:\Users\ASUS\AppData\Local\Temp\ipykernel_52416\3442198059.py:434: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) ins

In [14]:
Actualbill

array([[1123.92775109, 1832.00223428, 2028.94887412, 2311.72062995,
         944.09931092],
       [ 643.45702989,  781.34067916, 1111.60833497, 1008.29371203,
         612.81621895],
       [ 750.36933538,  911.16276439, 1063.02322512, 1173.84050554,
         714.63746227]])

In [15]:
actualprofit/(Retailer_Revenue_Household_actual+Retailer_Rev_actual)

array([0.10260457, 0.0402069 , 0.07727515])

In [16]:
actualprofit

array([1.50676354e+10, 5.30030444e+09, 1.08663523e+10])

In [17]:
Retailer_Revenue_Household_actual

array([4.58740030e+10, 2.55527345e+10, 3.13033563e+10])

In [18]:
Retailer_Rev_actual

array([1.00977503e+11, 1.06273013e+11, 1.09315626e+11])

In [19]:
Actualbill.sum(axis=0)*5/(15*12)

array([ 69.93761434,  97.9029355 , 116.76612317, 124.82930132,
        63.09869423])

In [20]:
MonthInv=np.zeros((1,5))
MonthInv2=Inv
for h in range(0,5):
    MonthInv[0,h]=(Inv[0,h]+Inv[1,h]+Inv[2,h])/(15*12)
print(MonthInv)

[[13.41302674 39.42754686 49.53519696 64.06976364 71.11088824]]


In [21]:
Pd.sum(axis=1)

0     3501.076585
1     7595.024294
2     9267.803243
3    12059.624405
4     5291.072306
dtype: float64

In [22]:
Bill0/12

array([[ 99.48090582, 162.15387649, 179.43913546, 203.58547443,
        132.30960474],
       [ 88.97624192, 151.07424409, 161.40266489, 185.80639192,
        118.63498922],
       [ 85.57423136, 145.29791366, 155.50491357, 179.60919084,
        114.09897514]])

In [23]:
300*12

3600

In [24]:
1519.35613963/5291.072306

0.2871546733366452

In [25]:
Retaileractual.tierrate

[0    59.553122
 dtype: float64,
 0    95.284995
 dtype: float64,
 0    139.949836
 dtype: float64,
 0    184.614678
 dtype: float64]

In [26]:
sumdf3actual

,House0,House1,House2,House3,House4
0,300.449455,749.261720,859.757828,1049.708298,264.482118
1,258.703267,634.590131,785.132931,953.095475,215.367698
2,166.718534,287.885142,514.475340,675.485233,10.993153
3,142.358293,226.697034,368.162044,507.883339,-19.046593
4,157.300229,262.811160,198.936008,310.802707,-32.797490
5,134.065528,212.868836,-9.112550,253.535836,-31.845988
6,81.367473,39.494750,-348.189482,12.647801,-336.746133
7,90.665753,194.981981,-275.946940,43.411542,-254.980751
8,147.488533,212.457498,-11.607156,206.252952,-125.960982
9,109.504487,53.213627,126.652763,196.769809,-243.098232


In [27]:
# # simple_array = [element[0] for element in HourlyLoad1]
# import datetime
colors = px.colors.qualitative.Plotly
fig = go.Figure()
# # start_date = datetime.datetime(2012, 1, 1, 23)
# # end_date = datetime.datetime(2022, 12, 31, 23)
# # date_rng = pd.date_range(start=start_date, end=end_date, freq='H')
# # timestamps = [datetime.datetime(2000, 1, 1, i) for i in range(0,23) ] * 365 * 20  # Repeated for 20 years

# fig.add_traces(go.Scatter(y = HourlyLoad2[0]*(0.0000001), mode = 'lines', line=dict(color=colors[3]), name='Load'))
# fig.add_traces(go.Scatter(y = Retaileractual.energy_proc_cost.astype(int)/10000000, mode = 'lines', line=dict(color=colors[4]), name='cost'))
# fig.add_traces(go.Scatter(y = WS_price/1000, mode = 'lines', line=dict(color=colors[2]), name='MSP'))
fig.add_traces(go.Scatter(y = Retailer_VC_actual[1], mode = 'lines', line=dict(color=colors[1]), name='Retail Price Actual k = 0'))
fig.add_traces(go.Scatter(y = Retailer_VC[1], mode = 'lines', line=dict(color=colors[0]), name='Retail Price Base'))
# fig.add_traces(go.Scatter(y = Retailer_VC_actual[1], mode = 'lines', line=dict(color=colors[2]), name='Retail Price Actual k = 1'))
# fig.add_traces(go.Scatter(y = Retailer_VC_actual[2], mode = 'lines', line=dict(color=colors[3]), name='Retail Price Actual k = 2'))

# fig.add_traces(go.Scatter(y = Retailer_VC_actual[0], mode = 'lines', line=dict(color=colors[0]), name='Retail Price Actual'))

# fig.update_layout(title='Market Settlement Price ($/MWh)')
# fig.update_xaxes(title_text='Date', tickformat='%Y-%m-%d')
# fig.update_yaxes(title_text="Price") # Y-axis label
# # pio.write_html(fig, 'Prices2.html')
# # fig.show()
              


In [28]:
Actualbill[k,:]/12

array([62.53077795, 75.93023037, 88.58526876, 97.82004213, 59.55312186])

In [29]:
Retailer.resepc.sum(axis=1).sum()

AttributeError: 'Energy_costs' object has no attribute 'resepc'

In [ ]:
np.average(Retailer_VC_actual[0])

In [ ]:
np.average(Retailer_VC_actual[1])

In [ ]:
np.average(Retailer_VC_actual[2])

In [ ]:
base.sum(axis=1)*1000

In [ ]:
np.maximum(0,(HourlyLoad1-cont_vol))

In [ ]:
csv_file_path = 'O3.csv'  # Specify the file path where you want to save the CSV file
a.to_csv(csv_file_path, index=False)  # Set index=False to exclude the DataFrame index from the CSV file

In [ ]:
data=(WSprice[(yy*hours):(hours*yyend)][1]/1000)

df = pd.DataFrame(data)

# Number of values per new column
values_per_column = 8760

# Calculate the number of new columns needed
num_new_columns = len(df) // values_per_column

# Reshape the data using NumPy
new_data = np.array_split(df, num_new_columns)

# Create a new DataFrame with the reshaped data
new_df = pd.DataFrame(new_data)

# Transpose the new DataFrame to have values in separate columns
new_df = new_df.T

# Rename the columns if needed (optional)
new_df.columns = [f'Column{i + 1}' for i in range(num_new_columns)]

# Now, new_df contains the values split into multiple columns


In [ ]:
import pandas as pd
import numpy as np

# Sample 3D array with shape (15, 8760, 1)
# data_3d = np.random.rand(15, 8760, 1)  # Replace this with your actual data

# Reshape into a 2D array with shape (15, 8760)
data_2d = new_data.reshape(15, -1)

# Create a DataFrame
df = pd.DataFrame(data_2d)

# Optionally, you can set column names if needed
column_names = [f'Column{i + 1}' for i in range(df.shape[1])]
df.columns = column_names
print(df.T)


In [ ]:
[581.081824*x1/12, 581.081824*x2/12, 581.081824*x3/12, 581.081824*x4/12]

In [ ]:
Pd*households

In [ ]:
HourlyResLoad1

In [ ]:
(HourlyResLoad1-HourlyInj1).sum(axis=1)